<h1 align=center><font size = 5>Applied Data Science Capstone</font></h1>
<h2 align=center><font size = 4>Week 3 Assignment, Part 2 <br>
    Segmenting and Clustering Neighborhoods in Toronto, Canada</font></h1>

## Overview:
In this Notebook the code will scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M and load 
the data into a pandas dataframe.  Rows with "Not assigned" value for Borough will be dropped.  Data will be grouped by Postal Code while 
retaining all Neighborhoods for the Postal Code.  If Neighborhood is not assigned, the 'Not assigned' value will be replaced with the value of Borough.  Latitude and Longitude will be retrieved and loaded into dataframe from csv file.  The two dataframes will be joined by postal code and merged into new dataframe. Unnecessary columns will be dropped from new dataframe.


### Import Pandas Library

In [2]:
import pandas as pd

### Load dataframe from Wikipedia page and set column names

In [3]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
df.columns = 'PostalCode','Borough','Neighborhood'
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Delete rows where column Borough has value of 'Not assigned' 

In [4]:
indexBorough = df[(df['Borough'] == 'Not assigned')].index
df.drop(indexBorough , axis=0, inplace=True)
df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


### Merge rows with duplicate PostalCode and Borough, retain Neighborhood value from merged rows and separate by comma

In [5]:
df = df.groupby(['PostalCode','Borough']).agg({'Neighborhood': ', '.join}).reset_index()
df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### If Neighborhood is 'Not assigned' set it to value of Borough and print postal code M7A for Queen's Park to validate

In [6]:
df.loc[df.Neighborhood == 'Not assigned','Neighborhood'] = df.Borough
print(df.loc[df['PostalCode'] == 'M7A'])

   PostalCode       Borough  Neighborhood
85        M7A  Queen's Park  Queen's Park


## Get number of rows and columns

In [7]:
df.shape

(103, 3)

### Load geo data from csv file into dataframe and label columns

In [8]:
df_geo = pd.read_csv('http://cocl.us/Geospatial_data')
df_geo.columns = 'GeoPC','Latitutude','Longitude'
df_geo.head()

,GeoPC,Latitutude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Merge dataframes to retrieve latitude and longitude for postal codes 

In [9]:
df_Toronto = pd.merge(df, df_geo, how='left', on=[df.PostalCode, df_geo.GeoPC])
df_Toronto.head()

,key_0,key_1,PostalCode,Borough,Neighborhood,GeoPC,Latitutude,Longitude
0,M1B,M1B,M1B,Scarborough,"Rouge, Malvern",M1B,43.806686,-79.194353
1,M1C,M1C,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",M1C,43.784535,-79.160497
2,M1E,M1E,M1E,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,M1G,M1G,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,M1H,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


### Drop unnecessary columns

In [10]:
df_Toronto.drop(['key_0','key_1','GeoPC'], axis=1, inplace=True)
df_Toronto.head()

,PostalCode,Borough,Neighborhood,Latitutude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
